In [1]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-1x1x1/"

In [2]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [3]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# Read wfndata
wfndata = wfn2py(dirname + "WFN_co.h5")
wfnqdata = wfn2py(dirname + "WFNq.h5")

# Read rho
rho = inp.read_rho(dirname + "RHO")

# Read vxc 
vxc = inp.read_vxc(dirname + "vxc.dat") 

In [4]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 1/1 [00:00<00:00, 3956.89it/s]


In [5]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )
       
        sort_order = sort_cryst_like_BGW(
            cryst = gkspc.g_cryst, key_array = gkspc.gk_norm2  
        )

        print(f"indices_gspace_sorted {indices_gspace_sorted}")
        print(f"sort_order {sort_order}")

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv0 = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)
        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv0, indices_gspace_sorted)

        epsinv0 = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)
        epsinv1 = reorder_2d_matrix_sorted_gvecs(epsinv0, sort_order)

        epsilon.l_epsinv[i_q] = epsinv

        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))
        std_eps2 = np.std(epsref - epsinv1) / np.sqrt(np.prod(list(epsinv0.shape)))

        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")

        if np.abs(std_eps2) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps2}, for i_q:{i_q}")

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 1/1 [00:00<00:00, 273.94it/s]

indices_gspace_sorted [ 0 14  2 11  5  3  7  1 10 12  6 13  9  4  8]
sort_order [ 0 14  2 11  5  3  7  1 10 12  6 13  9  4  8]
Standard deviation exceeded 1e-16 tolerance: 1.9894890166455426e-15, for i_q:0
Standard deviation exceeded 1e-16 tolerance: 1.9894890166455426e-15, for i_q:0


In [6]:
# Construct the Charge Matrix Class
from mtxel import ChargeMtxEL

cmtxclass = ChargeMtxEL.from_BGW(
    wfndata = wfndata,
    epsinp = epsinp,
    sigmainp = sigmainp,
    num_bands_val = 1,
    num_bands_con = 1,
)

In [7]:
from read_mtxel import *
bgw_arrays = parse_file(dirname + "kernel.out")

mccp_BGW = bgw_arrays.get('mccp', None)
mvvp_BGW = bgw_arrays.get('mvvp', None)
mvc_BGW = bgw_arrays.get('mvc', None)
mvpcp_BGW = bgw_arrays.get('mvpcp', None)

In [8]:
print(f"Shape of mccp_BGW is {mccp_BGW.shape}")
print(f"Shape of mvvp_BGW is {mvvp_BGW.shape}")
print(f"Shape of mvc_BGW is {mvc_BGW.shape}")
print(f"Shape of mvpcp_BGW is {mvpcp_BGW.shape}")

Shape of mccp_BGW is (27, 1, 1, 1)
Shape of mvvp_BGW is (27, 1, 1, 1)
Shape of mvc_BGW is (27, 1, 1, 1)
Shape of mvpcp_BGW is (27, 1, 1, 1)


In [9]:
is_equal_mvc_mvpcp = np.allclose(mvc_BGW, mvpcp_BGW)
print(f"Is mvc and mvpcp equal? {is_equal_mvc_mvpcp}")

Is mvc and mvpcp equal? True


In [10]:
mccp_m2 = cmtxclass.matrix_element('mccp', 0, 0)
mccp_m2 = mccp_m2[0, 0, :]
mccp_BGW = mccp_BGW[:, 0, 0, 0]

are_mccp_equal = np.allclose(mccp_BGW, mccp_m2)
print(f"Are mccp_BGW and mccp_m2 equal? {are_mccp_equal}")

Are mccp_BGW and mccp_m2 equal? True


In [11]:
mvc_m2 = cmtxclass.matrix_element('mvc', 0)
mvc_m2 = mvc_m2[0, 0, :]
mvc_BGW = mvc_BGW[:, 0, 0, 0]

are_mvc_equal = np.allclose(mvc_BGW, mvc_m2)
print(f"Are mvc_BGW and mvc_m2 equal? {are_mvc_equal}")

Are mvc_BGW and mvc_m2 equal? True


In [12]:
mvvp_m2 = cmtxclass.matrix_element('mvvp', 0, 0)
mvvp_m2 = mvvp_m2[0, 0, :]
mvvp_BGW = mvvp_BGW[:, 0, 0, 0]

are_mvvp_equal = np.allclose(mvvp_BGW, mvvp_m2)
print(f"Are mvvp_BGW and mvvp_m2 equal? {are_mvvp_equal}")

Are mvvp_BGW and mvvp_m2 equal? True
